# Xgboost

Ref: https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import KFold, GridSearchCV, cross_validate
from spaceship_titanic import feature_enginnering as fe

In [6]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [7]:
def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = fe.fillna_homeplanet_and_destination(df)
    df = fe.fill_with_0_people_with_no_other_wastes(df)
    df = fe.fill_0_wastes_people_cryosleep(df)
    df = fe.fill_with_non_0_median(df)
    df = fe.fill_cryosleep(df)
    df = fe.feature_inputer(df, to_mode=[], to_median=['Age'])
    df = fe.cabin_inputer(df)
    df = fe.vip_knn_input(df)
    df['has_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) != 0
    df = fe.outliers_to_log(df)
    df = fe.dtype_memory_reducer(df)
    return df
def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df['Deck'] = df['Deck'].map(dict(zip(['A', 'B', 'C', 'D', 'E', 'F', 'G'],[1,2,3,4,5,6,7])))
    df = pd.get_dummies(df,columns=['HomePlanet','Destination'],drop_first=True)
    return df

def train_test_treatments(df, full_df):
    df['is_alone'] = fe.calculate_groupsize(df.index, full_df.index) == 1
    return df

In [8]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)
df_train = train_test_treatments(df_train, pd.concat([df_train,df_test]))

In [9]:
X_train, X_val, y_train, y_val = train_test_split(
    df_train.drop('Transported',axis=1), df_train['Transported'],
    test_size=0.2,
    random_state=42
)

## Tuning

In [18]:
params = {
 'max_depth': [2, 3, 4],
 'learning_rate': [0.1,0.5],
 'n_estimators': [100, 200],
 'subsample':[0.7,0.8, 1],
 'colsample_bytree': [0.7, 0.8, 1],
}

In [22]:
grid = GridSearchCV(
    xgb.XGBClassifier(early_stopping_rounds=10),
    param_grid=params,
    scoring='accuracy',
    cv=KFold(5, shuffle=True, random_state=42),
    verbose=3,
    n_jobs=2
    return_train_score=True
)
grid.fit(X_train, y_train, eval_set = (X_val,y_val))

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[0]	validation_0-logloss:0.66658
[1]	validation_0-logloss:0.64217
[2]	validation_0-logloss:0.62478
[3]	validation_0-logloss:0.60964
[4]	validation_0-logloss:0.59529
[5]	validation_0-logloss:0.58400
[6]	validation_0-logloss:0.57296
[7]	validation_0-logloss:0.56370
[8]	validation_0-logloss:0.55653
[9]	validation_0-logloss:0.54987
[10]	validation_0-logloss:0.54385
[11]	validation_0-logloss:0.53858
[12]	validation_0-logloss:0.53312
[13]	validation_0-logloss:0.52888
[14]	validation_0-logloss:0.52397
[15]	validation_0-logloss:0.52011
[16]	validation_0-logloss:0.51684
[17]	validation_0-logloss:0.51384
[18]	validation_0-logloss:0.51087
[19]	validation_0-logloss:0.50727
[20]	validation_0-logloss:0.50444
[21]	validation_0-logloss:0.50179
[22]	validation_0-logloss:0.49938
[23]	validation_0-logloss:0.49759
[24]	validation_0-logloss:0.49647
[25]	validation_0-logloss:0.49414
[26]	validation_0-logloss:0.49256
[27]	validation_0-logloss:0.4

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-logloss:0.46778
[51]	validation_0-logloss:0.46718
[52]	validation_0-logloss:0.46646
[53]	validation_0-logloss:0.46578
[54]	validation_0-logloss:0.46517
[55]	validation_0-logloss:0.46493
[56]	validation_0-logloss:0.46438
[57]	validation_0-logloss:0.46384
[58]	validation_0-logloss:0.46337
[59]	validation_0-logloss:0.46274
[60]	validation_0-logloss:0.46272
[61]	validation_0-logloss:0.46226
[62]	validation_0-logloss:0.46192
[63]	validation_0-logloss:0.46161
[64]	validation_0-logloss:0.46126
[65]	validation_0-logloss:0.46112
[66]	validation_0-logloss:0.46085
[67]	validation_0-logloss:0.46064
[68]	validation_0-logloss:0.46047
[69]	validation_0-logloss:0.46012
[70]	validation_0-logloss:0.45972
[71]	validation_0-logloss:0.45940
[72]	validation_0-logloss:0.45905
[73]	validation_0-logloss:0.45868
[74]	validation_0-logloss:0.45846
[75]	validation_0-logloss:0.45810
[76]	validation_0-logloss:0.45765
[77]	validation_0-logloss:0.45711
[78]	validation_0-logloss:0.45650
[79]	validatio

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[54]	validation_0-logloss:0.46699
[55]	validation_0-logloss:0.46644
[56]	validation_0-logloss:0.46614
[57]	validation_0-logloss:0.46510
[58]	validation_0-logloss:0.46424
[59]	validation_0-logloss:0.46402
[60]	validation_0-logloss:0.46373
[61]	validation_0-logloss:0.46332
[62]	validation_0-logloss:0.46289
[63]	validation_0-logloss:0.46261
[64]	validation_0-logloss:0.46216
[65]	validation_0-logloss:0.46207
[66]	validation_0-logloss:0.46192
[67]	validation_0-logloss:0.46154
[68]	validation_0-logloss:0.46124
[69]	validation_0-logloss:0.46090
[70]	validation_0-logloss:0.46027
[71]	validation_0-logloss:0.45997
[72]	validation_0-logloss:0.45973
[73]	validation_0-logloss:0.45902
[74]	validation_0-logloss:0.45868
[75]	validation_0-logloss:0.45821
[76]	validation_0-logloss:0.45820
[77]	validation_0-logloss:0.45775
[78]	validation_0-logloss:0.45740
[79]	validation_0-logloss:0.45713
[80]	validation_0-logloss:0.45688
[81]	validation_0-logloss:0.45698
[82]	validation_0-logloss:0.45672
[83]	validatio

...

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-logloss:0.44331
[7]	validation_0-logloss:0.44057
[8]	validation_0-logloss:0.44053
[9]	validation_0-logloss:0.43994
[10]	validation_0-logloss:0.43476
[11]	validation_0-logloss:0.43441
[12]	validation_0-logloss:0.43441
[13]	validation_0-logloss:0.43403
[14]	validation_0-logloss:0.43379
[15]	validation_0-logloss:0.43180
[16]	validation_0-logloss:0.43146
[17]	validation_0-logloss:0.43117
[18]	validation_0-logloss:0.43216
[19]	validation_0-logloss:0.43127
[20]	validation_0-logloss:0.43113
[21]	validation_0-logloss:0.43195
[22]	validation_0-logloss:0.43276
[23]	validation_0-logloss:0.43237
[24]	validation_0-logloss:0.43219
[25]	validation_0-logloss:0.43260
[26]	validation_0-logloss:0.43309
[27]	validation_0-logloss:0.43295
[28]	validation_0-logloss:0.43290
[29]	validation_0-logloss:0.43428
[30]	validation_0-logloss:0.43554
[31]	validation_0-logloss:0.43773
[32]	validation_0-logloss:0.43691
[33]	validation_0-logloss:0.43739
[34]	validation_0-logloss:0.43720
[35]	validation_0-

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[37]	validation_0-logloss:0.45726
[38]	validation_0-logloss:0.45639
[39]	validation_0-logloss:0.45554
[40]	validation_0-logloss:0.45519
[41]	validation_0-logloss:0.45472
[42]	validation_0-logloss:0.45419
[43]	validation_0-logloss:0.45401
[44]	validation_0-logloss:0.45325
[45]	validation_0-logloss:0.45239
[46]	validation_0-logloss:0.45214
[47]	validation_0-logloss:0.45163
[48]	validation_0-logloss:0.45108
[49]	validation_0-logloss:0.45095
[50]	validation_0-logloss:0.45080
[51]	validation_0-logloss:0.45043
[52]	validation_0-logloss:0.44998
[53]	validation_0-logloss:0.44968
[54]	validation_0-logloss:0.44953
[55]	validation_0-logloss:0.44951
[56]	validation_0-logloss:0.44763
[57]	validation_0-logloss:0.44718
[58]	validation_0-logloss:0.44670
[59]	validation_0-logloss:0.44625
[60]	validation_0-logloss:0.44585
[61]	validation_0-logloss:0.44543
[62]	validation_0-logloss:0.44538
[63]	validation_0-logloss:0.44531
[64]	validation_0-logloss:0.44509
[65]	validation_0-logloss:0.44511
[66]	validatio

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     int...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.7, 0.8, 1],
                         'learning_rate': [0.1, 0.5], 'max_depth': [2, 3, 4],
                         'n_estimators': [100, 200],
                         'subsample': [0.7, 0.8, 1]},
             return_train_score=True, scoring='accuracy', verbose=3)

In [23]:
grid.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 200,
 'subsample': 0.8}

In [24]:
pd.DataFrame(grid.cv_results_)[['mean_train_score','std_train_score','mean_test_score','std_test_score']].loc[grid.best_index_]

mean_train_score    0.822117
std_train_score     0.003849
mean_test_score     0.802128
std_test_score      0.006443
Name: 46, dtype: float64

In [29]:
model = grid.best_estimator_
model.fit(X_train, y_train, early_stopping_rounds=20, eval_set = [(X_val,y_val)] )

[0]	validation_0-logloss:0.66452
[1]	validation_0-logloss:0.63743
[2]	validation_0-logloss:0.61500
[3]	validation_0-logloss:0.59620
[4]	validation_0-logloss:0.57963
[5]	validation_0-logloss:0.56591
[6]	validation_0-logloss:0.55321
[7]	validation_0-logloss:0.54305
[8]	validation_0-logloss:0.53521
[9]	validation_0-logloss:0.52848
[10]	validation_0-logloss:0.52122
[11]	validation_0-logloss:0.51369
[12]	validation_0-logloss:0.50761
[13]	validation_0-logloss:0.50234
[14]	validation_0-logloss:0.49753
[15]	validation_0-logloss:0.49460
[16]	validation_0-logloss:0.49039
[17]	validation_0-logloss:0.48684
[18]	validation_0-logloss:0.48424
[19]	validation_0-logloss:0.48056
[20]	validation_0-logloss:0.47775
[21]	validation_0-logloss:0.47537
[22]	validation_0-logloss:0.47311
[23]	validation_0-logloss:0.47117
[24]	validation_0-logloss:0.46931
[25]	validation_0-logloss:0.46810
[26]	validation_0-logloss:0.46718
[27]	validation_0-logloss:0.46563
[28]	validation_0-logloss:0.46431
[29]	validation_0-loglos

c:\users\gonca\desktop\trabalho\resources\kaggle\.venv\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[38]	validation_0-logloss:0.45639
[39]	validation_0-logloss:0.45554
[40]	validation_0-logloss:0.45519
[41]	validation_0-logloss:0.45472
[42]	validation_0-logloss:0.45419
[43]	validation_0-logloss:0.45401
[44]	validation_0-logloss:0.45325
[45]	validation_0-logloss:0.45239
[46]	validation_0-logloss:0.45214
[47]	validation_0-logloss:0.45163
[48]	validation_0-logloss:0.45108
[49]	validation_0-logloss:0.45095
[50]	validation_0-logloss:0.45080
[51]	validation_0-logloss:0.45043
[52]	validation_0-logloss:0.44998
[53]	validation_0-logloss:0.44968
[54]	validation_0-logloss:0.44953
[55]	validation_0-logloss:0.44951
[56]	validation_0-logloss:0.44763
[57]	validation_0-logloss:0.44718
[58]	validation_0-logloss:0.44670
[59]	validation_0-logloss:0.44625
[60]	validation_0-logloss:0.44585
[61]	validation_0-logloss:0.44543
[62]	validation_0-logloss:0.44538
[63]	validation_0-logloss:0.44531
[64]	validation_0-logloss:0.44509
[65]	validation_0-logloss:0.44511
[66]	validation_0-logloss:0.44457
[67]	validatio

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=3, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=200, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [30]:
scores = cross_validate(
    model,
    X_val,
    y_val,
    cv=KFold(5, shuffle=True, random_state=42),
    scoring = "accuracy",
    return_train_score=True
)
results = {
    'train_score_avg':np.mean(scores['train_score']),
    'train_score_std':np.std(scores['train_score']),
    'test_score_avg':np.mean(scores['test_score']),
    'test_score_std':np.std(scores['test_score']),
}
results

{'train_score_avg': 0.861559119793087,
 'train_score_std': 0.00469320453840181,
 'test_score_avg': 0.7745933949451788,
 'test_score_std': 0.018200276303289197}

# Submission

In [31]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test = train_test_treatments(X_test, pd.concat([df_train,df_test]))
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,has_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,is_alone
PassengerId,,,,,,,,,,,,,,,
0013_01,True,27.0,False,0.0,0.000000,0.0,0.00000,0.0,7,False,0,0,0,1,True
0018_01,False,19.0,False,0.0,2.302585,0.0,7.94591,0.0,6,True,0,0,0,1,True


In [32]:
y_test = model.predict(X_test)
submission = pd.Series(y_test, index=[X_test.index]).astype(bool).to_frame('Transported')
submission.head()

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True


In [33]:
submission.to_csv('data/submission_xgboost_2022-11-07.csv')